# GPU硬件加速

In [2]:
from __future__ import annotations
import numpy as np
import tvm
from tvm import te, relax, IRModule
from tvm.script import relax as R
from tvm.script import tir as T

### 6.1.3 GPU体系结构
处理器：GPU ---> SM ---> Core \
存储：global_memory ---> shared memory(SM) ---> register(core) \
调度单位：SIMT(thread blocks) ---> thread 映射到 core / thread block 映射到SM \

warp schedule: 32

In [3]:
@tvm.script.ir_module
class MyModuleVecAdd():
    @T.prim_func
    def main(A: T.Buffer[(1024,), 'float32'],
             B: T.Buffer[(1024,), 'float32'],
             C: T.Buffer[(1024,), 'float32']):
        T.func_attr({"global_symbol": "main", "tir.noalias":True})
        for i in T.grid(1024):
            with T.block("C"):
                vi = T.axis.spatial((0, 1024), i)
                C[vi] = A[vi] + B[vi]


In [4]:
sch = tvm.tir.Schedule(MyModuleVecAdd)
block_C = sch.get_block('C', 'main')
i, = sch.get_loops(block_C)
i0, i1 = sch.split(i, [None, 128])
sch.mod.show()

/staff/qiaoliang/anaconda3/envs/MLC/lib/python3.8/site-packages/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/staff/qiaoliang/anaconda3/envs/MLC/bin/python -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


#### 6.1.3.1 GPU线程块

In [5]:
sch.bind(i0, "blockIdx.x")
sch.bind(i1, 'threadIdx.x')
sch.mod.show()

/staff/qiaoliang/anaconda3/envs/MLC/lib/python3.8/site-packages/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/staff/qiaoliang/anaconda3/envs/MLC/bin/python -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


#### 6.1.3.2 在GPU上构建和运行TensorIR函数

In [6]:
rt_mod = tvm.build(sch.mod, target='cuda')
A_np = np.random.uniform(size=(1024,)).astype("float32")
B_np = np.random.uniform(size=(1024,)).astype('float32')
A_nd = tvm.nd.array(A_np, tvm.cuda(0))
B_nd = tvm.nd.array(B_np, tvm.cuda(0))
C_nd = tvm.nd.empty((1024,), dtype='float32', device=tvm.cuda(0))
print(type(C_nd))
rt_mod['main'](A_nd, B_nd, C_nd)
print(A_nd)
print(B_nd)
print(C_nd, type(C_nd))

<class 'tvm.runtime.ndarray.NDArray'>
[0.14235647 0.45518884 0.7623538  ... 0.45710036 0.24382155 0.33414462]
[0.19423197 0.5293532  0.2514951  ... 0.6796483  0.8336089  0.6925215 ]
[0.33658844 0.984542   1.0138489  ... 1.1367487  1.0774305  1.0266662 ] <class 'tvm.runtime.ndarray.NDArray'>


### 示例：窗口求和

In [7]:
@tvm.script.ir_module
class MyModuleWindowSun():
    @T.prim_func
    def main(A: T.Buffer[(1026,), 'float32'],
             B: T.Buffer[(1024,), 'float32']):
        T.func_attr({"global_symbol": "main", "tir.noalias": True})
        for i in T.grid(1024):
            with T.block('B'):
                vi = T.axis.spatial((0, 1024), i)
                B[vi] = A[vi] + A[vi+1] + A[vi+2]

In [8]:
sch = tvm.tir.Schedule(MyModuleWindowSun)
block_B = sch.get_block('B', 'main')
i, = sch.get_loops(block_B)
i0, i1 = sch.split(i, [None, 128])
sch.bind(i0, "blockIdx.x")
sch.bind(i1, "threadIdx.x")
sch.mod.show()

/staff/qiaoliang/anaconda3/envs/MLC/lib/python3.8/site-packages/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/staff/qiaoliang/anaconda3/envs/MLC/bin/python -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


每个 GPU 线程块都包含所有线程都可以在块内访问的共享内存 (shared memory)。 我们使用cache_read添加一个中间阶段，将部分数据（下面绿色）缓存到共享内存上。 缓存完成后，线程可以从共享内存中读取。

上述例子：每个thread block中存在128个线程，需要读取global memory 384次(3 * 128)，使用共享内存，只需要读取130次

In [9]:
A_shared = sch.cache_read(block_B, 0, 'shared')
sch.mod.show()

/staff/qiaoliang/anaconda3/envs/MLC/lib/python3.8/site-packages/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/staff/qiaoliang/anaconda3/envs/MLC/bin/python -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


In [10]:
sch.compute_at(A_shared, i1)

In [11]:

sch.mod.show()

/staff/qiaoliang/anaconda3/envs/MLC/lib/python3.8/site-packages/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/staff/qiaoliang/anaconda3/envs/MLC/bin/python -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


因为内存是跨线程共享的，所以我们需要重新拆分循环并将获取过程的内部迭代器绑定到线程索引上。这种技术称为 **cooperative fetching**，其中多个线程一起工作以将数据带到共享内存中。下面的读取过程会与之前不同。
```python
A_shared = T.alloc_buffer((1026,), scope="shared")
for i_0 in T.thread_binding(8, thread="blockIdx.x"):
    for i_1 in T.thread_binding(128, thread="threadIdx.x"):
        for ax0 in range(130):
            with T.block("A_shared"):
                v0 = T.axis.spatial(1026, i_0 * 128 + ax0)   ## 会造成重复读取，每个线程读取130个数据
                T.reads(A[v0])
                T.writes(A_shared[v0])
                A_shared[v0] = A[v0]
                
```

In [12]:
ax = sch.get_loops(A_shared)[-1]
ax0, ax1 = sch.split(ax, [None, 128])
sch.bind(ax1, "threadIdx.x")

In [13]:
sch.mod.show()

/staff/qiaoliang/anaconda3/envs/MLC/lib/python3.8/site-packages/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/staff/qiaoliang/anaconda3/envs/MLC/bin/python -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


我们可以检查相应的底层代码（CUDA 中）。 生成的代码包含两部分：
1. 在主机 (CPU) 上的调用 GPU 程序的部分；
2. 相应计算的 CUDA 内核。
我们可以使用以下代码打印出 CUDA 内核。 我们仍然需要主机和内核代码来运行程序，因此它只是一种快速检查最终代码生成结果的方法。

**值得注意的是，构建过程会自动压缩共享内存阶段以使用线程块中使用的最小区域**

In [14]:
rt_mod = tvm.build(sch.mod, target="cuda")
print(rt_mod.imported_modules[0].get_source())


#ifdef _WIN32
  using uint = unsigned int;
  using uchar = unsigned char;
  using ushort = unsigned short;
  using int64_t = long long;
  using uint64_t = unsigned long long;
#else
  #define uint unsigned int
  #define uchar unsigned char
  #define ushort unsigned short
  #define int64_t long long
  #define uint64_t unsigned long long
#endif
extern "C" __global__ void __launch_bounds__(128) main_kernel0(float* __restrict__ A, float* __restrict__ B) {
  __shared__ float A_shared[130];
  for (int ax0_0 = 0; ax0_0 < 2; ++ax0_0) {
    if (((ax0_0 * 64) + (((int)threadIdx.x) >> 1)) < 65) {
      A_shared[((ax0_0 * 128) + ((int)threadIdx.x))] = A[(((((int)blockIdx.x) * 128) + (ax0_0 * 128)) + ((int)threadIdx.x))];
    }
  }
  __syncthreads();
  B[((((int)blockIdx.x) * 128) + ((int)threadIdx.x))] = ((A_shared[((int)threadIdx.x)] + A_shared[(((int)threadIdx.x) + 1)]) + A_shared[(((int)threadIdx.x) + 2)]);
}




In [15]:
rt_mod = tvm.build(sch.mod, target="metal")
print(rt_mod.imported_modules[0].get_source())

// Function: main_kernel0
#include <metal_stdlib>
using namespace metal;

union __TVMArgUnion {
 int v_int[2];
};

kernel void main_kernel0(  device float* A [[ buffer(0) ]],
  device float* B [[ buffer(1) ]],
  uint blockIdx [[threadgroup_position_in_grid]],
  uint threadIdx [[thread_position_in_threadgroup]]
) {
  threadgroup float A_shared[130];
  for (int ax0_0 = 0; ax0_0 < 2; ++ax0_0) {
    if (((ax0_0 * 64) + (((int)threadIdx) >> 1)) < 65) {
      A_shared[((ax0_0 * 128) + ((int)threadIdx))] = A[(((((int)blockIdx) * 128) + (ax0_0 * 128)) + ((int)threadIdx))];
    }
  }
  threadgroup_barrier(mem_flags::mem_threadgroup);
  B[((((int)blockIdx) * 128) + ((int)threadIdx))] = ((A_shared[((int)threadIdx)] + A_shared[(((int)threadIdx) + 1)]) + A_shared[(((int)threadIdx) + 2)]);
}




### 6.1.5 矩阵乘法

In [16]:
@tvm.script.ir_module
class MyModuleMatmul():
    @T.prim_func
    def main(A: T.Buffer[(1024, 1024), 'float32'],
             B: T.Buffer[(1024, 1024), 'float32'],
             C: T.Buffer[(1024, 1024), 'float32']):
        T.func_attr({"global_symbol": "main", "tir.noalias": True})
        for i, j, k in T.grid(1024, 1024, 1024):
            with T.block('C'):
                vi, vj, vk = T.axis.remap("SSR", [i, j, k])
                with T.init():
                    C[vi, vj] = T.float32(0)
                C[vi, vj] = C[vi, vj] + A[vi, vk] * B[vk, vj]

In [17]:
MyModuleMatmul.show()

/staff/qiaoliang/anaconda3/envs/MLC/lib/python3.8/site-packages/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/staff/qiaoliang/anaconda3/envs/MLC/bin/python -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


In [18]:
record ={}
dev = tvm.cuda(0)
A_np = np.random.uniform(size=(1024, 1024)).astype('float32')
B_np = np.random.uniform(size=(1024, 1024)).astype('float32')
A_nd = tvm.nd.array(A_np, device=dev)
B_nd = tvm.nd.array(B_np, device=dev)
C_nd = tvm.nd.empty((1024, 1024), dtype='float32', device=dev)

#### 6.1.5.1 本地存储分块

我们可以进行循环拆分，来增加整体内存复用。特别是，我们引入了局部切分，这样我们只需要从 A 和 B 加载一次条形数据（上图中的灰色部分），然后使用它们来执行 $V * V$矩阵乘法结果

In [33]:
def blocking(sch: tvm.tir.Schedule,
    tile_block_x,
    tile_block_y,
    tile_thread_x,
    tile_thread_y,
    tile_k):
    block_C = sch.get_block("C", 'main')
    C_local = sch.cache_write(block_C, 0, 'local')  # 2
    i, j, k = sch.get_loops(block_C)
    
    i0, i1, i2 = sch.split(i, [None, tile_block_x, tile_thread_x])
    j0, j1, j2 = sch.split(j, [None, tile_block_y, tile_thread_y])
    k0, k1 = sch.split(k, [None, tile_k])
    sch.unroll(k1)
    sch.reorder(i0, j0, i1, j1, k0, k1, i2, j2)
    sch.reverse_compute_at(C_local, j1)  # 2
    sch.bind(i0, "blockIdx.x")
    sch.bind(j0, "blockIdx.y")
    sch.bind(i1, "threadIdx.x")
    sch.bind(j1, "threadIdx.y")

    sch.decompose_reduction(block_C, k0)
    return sch

sch = tvm.tir.Schedule(MyModuleMatmul)

sch = blocking(sch, 8, 8, 8, 8, 4)

sch.mod.show()
rt_lib = tvm.build(sch.mod, target='cuda')

num_flop = 2 * 1024 * 1024 * 1024
evaluator = rt_lib.time_evaluator('main', dev, 100)
version_ = 'GEMM-3'
gflops = num_flop / evaluator(A_nd, B_nd, C_nd).mean / 1e9
print("%s: %f GFLOPS" % (version_, gflops))
np.testing.assert_allclose(C_nd.numpy(), A_np @ B_np, rtol=1e-5)
record[version_] = {"sch": sch, "gflops": gflops}


/staff/qiaoliang/anaconda3/envs/MLC/lib/python3.8/site-packages/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/staff/qiaoliang/anaconda3/envs/MLC/bin/python -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


GEMM-3: 3750.875499 GFLOPS


In [34]:
record

{'GEMM-1': {'sch': tir.Schedule(0x55983d7d23e8), 'gflops': 56.87070155683453},
 'GEMM-2': {'sch': tir.Schedule(0x55983dfe0468), 'gflops': 3739.8387250840606},
 'GEMM-3': {'sch': tir.Schedule(0x55983d8711f8), 'gflops': 3750.8754985602113}}

### 6.1.6 共享内存分块

In [49]:
def cache_read_and_coop_fetcch(sch: tvm.tir.Schedule,
                    block, read_idx, read_loc):
    # cache_read
    read_cache = sch.cache_read(block=block, read_buffer_index=read_idx, storage_scope='shared')
    sch.compute_at(read_cache, read_loc)
    # coo_fetch
    i, j = sch.get_loops(read_cache)[-2:]
    fused = sch.fuse(i, j)
    _, tx, vec = sch.split(fused, [None, 64, 4])
    sch.bind(tx, "threadIdx.x")
    sch.vectorize(vec)
    



def blocking_with_shared(sch: tvm.tir.Schedule,
            tile_block_x,
            tile_block_y,
            tile_thread_x,
            tile_thread_y,
            tile_k):
    block_C = sch.get_block('C', 'main')
    C_local = sch.cache_write(block_C, 0, 'local')
    i, j, k = sch.get_loops(block_C)

    i0, i1, i2 = sch.split(i, [None, tile_block_x, tile_thread_x])
    j0, j1, j2 = sch.split(j, [None, tile_block_y, tile_thread_y])
    k0, k1 = sch.split(k, [None, tile_k])

    sch.reorder(i0, j0, i1, j1, k0, k1, i2, j2)
    tx = sch.fuse(i1, j1)
    sch.reverse_compute_at(C_local, tx)

    sch.bind(i0, 'blockIdx.x')
    sch.bind(j0, 'blockIdx.y')
    sch.bind(tx, 'threadIdx.x')

    cache_read_and_coop_fetcch(sch, block_C, 0, k0)
    cache_read_and_coop_fetcch(sch, block_C, 1, k0)

    # sch.decompose_reduction(block_C, k0)
    return sch
    
sch = tvm.tir.Schedule(MyModuleMatmul)
sch = blocking_with_shared(sch, 8, 8, 8, 8, 8)
sch.mod.show()
    

/staff/qiaoliang/anaconda3/envs/MLC/lib/python3.8/site-packages/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/staff/qiaoliang/anaconda3/envs/MLC/bin/python -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


In [53]:
rt_mod = tvm.build(sch.mod, target='cuda')
dev = tvm.cuda(0)
evaluator = rt_mod.time_evaluator('main', dev=dev, number=100)
gflops = (num_flop / evaluator(A_nd, B_nd, C_nd).mean / 1e9)
print("GEMM-Blocking: %f GFLOPS" % gflops)
np.testing.assert_allclose(C_nd.numpy(), A_np @ B_np, rtol=1e-5)
record['GEMM-4'] = {"sch": sch, "gflops": gflops}


GEMM-Blocking: 5768.538183 GFLOPS


In [54]:
record

{'GEMM-1': {'sch': tir.Schedule(0x55983d7d23e8), 'gflops': 56.87070155683453},
 'GEMM-2': {'sch': tir.Schedule(0x55983dfe0468), 'gflops': 3739.8387250840606},
 'GEMM-3': {'sch': tir.Schedule(0x55983d8711f8), 'gflops': 3750.8754985602113},
 'GEMM-blocking': {'sch': tir.Schedule(0x55983d744ad8),
  'gflops': 5804.622168657413},
 'GEMM-4': {'sch': tir.Schedule(0x55983d744ad8), 'gflops': 5768.538182735198}}

### 6.1.7 利用自动程序优化

In [95]:
from tvm import meta_schedule as ms

tuned_record = ms.tune_tir(MyModuleMatmul, target="cuda --max_threads_per_block=1024 --max_shared_memory_per_block=49152",
                            work_dir="./tune_tmp/",
                            task_name='main',
                            max_trials_global=64,
                            num_trials_per_iter=64)

2023-02-10 20:40:33 [INFO] [task_scheduler.cc:260] Task #0 has finished. Remaining task(s): 0


,Name,FLOP,Weight,Speed (GFLOPS),Latency (us),Weighted Latency (us),Trials,Done
0,main,2147483648,1,8435.4300,254.5790,254.5790,64,Y


2023-02-10 20:40:33 [DEBUG] [task_scheduler.cc:318] 
 ID | Name |       FLOP | Weight | Speed (GFLOPS) | Latency (us) | Weighted Latency (us) | Trials | Done 
---------------------------------------------------------------------------------------------------------
  0 | main | 2147483648 |      1 |      8435.4300 |     254.5790 |              254.5790 |     64 |    Y 
---------------------------------------------------------------------------------------------------------
Total trials: 64
Total latency (us): 254.579



In [96]:
from tvm.meta_schedule.tir_integration import compile_tir
new_sch = compile_tir(tuned_record, MyModuleMatmul, target='cuda')

In [98]:
# new_sch.mod.show()
rt_lib_autotvm = tvm.build(new_sch.mod, target='cuda')
evaluator = rt_lib_autotvm.time_evaluator('main', dev, number=1000)
gflops = (num_flop / evaluator(A_nd, B_nd, C_nd).mean / 1e9)
print("GEMM-5: %f GFLOPS" % gflops)
np.testing.assert_allclose(C_nd.numpy(), A_np @ B_np, rtol=1e-5)
record['GEMM-5'] = {"sch": new_sch, "gflops": gflops}

GEMM-5: 8309.963287 GFLOPS


### 6.1.8 小结
本章研究 MLC 的另一个维度，即我们如何变换我们的程序以实现硬件加速。MLC 过程帮助我们将输入模型连接到不同的 GPU 编程模型和环境。 我们还将在接下来的章节中访问更多专业硬件加速主题。
1. 典型的 GPU 包含两级层次结构。 每个线程由（在 CUDA 术语中）threadIdx.x 和 blockIdx.x 索引（也可以有多个维度索引，但它们可以融合为一个）。
2. 共享内存有助于缓存同一块内的线程中常用的数据。
3. 在 GPU 优化期间鼓励内存重用。